<h1 align="center">Aggregate Planning</h1>


OM Inc. sells various types of widgets, and wants to develop a monthly aggregate production plan for the next six months, considering an "aggregate" widget, given the following information:

Expected aggregate demand for widgets for the next six months:

Month 1: 1,200  
Month 2: 1,500  
Month 3: 1,800  
Month 4: 1,900  
Month 5: 2,100  
Month 6: 2,200  

A widget sells for \$200, and has a back-order cost of \$10 per unit per month. The inventory holding cost is \$5 per unit per month. Starting inventory for month 1 is 200 units. It takes 3 worker-hours, and \$10 worth of raw materials, to produce one widget.

At the start of month 1 the company has 30 workers. Workers are paid for 160 regular-time hours per month, at a rate of \$20 an hour. Each worker can work up to 20 overtime hours per month, at a rate of \$30 an hour, layoffs cost \$2000 per worker, and new hires cost \$1000 for initial processing and training.

The company wants to meet demand while minimizing the costs of doing so over the six-month planning horizon.

**Parameters:**  
$T$: set of months (1 to 6), for the remainder ($t \in T$)  
$w_0$: number of workers at the start of month 1 ($w_0 = 30$)  
$b_0$: back-orders at the start of month 1 ($b_0=0$)  
$i_0$: inventory held the start of month 1 ($i_0 =200$)  
$D_t$: demand for month $t$  
$S$: regular monthly pay rate per worker ($S=160*20$)  
$O$: overtime pay rate per hour ($O=30$)  
$H$: cost of hiring a new worker  ($H=1000$)  
$L$: cost of laying off a worker ($L=2000$)  
$R$: regular time hours per worker per month ($R=160$)  
$M$: maximum overtime hours per worker per month ($M=20$)  
$C$: raw material cost per widget ($C=10$)  
$I$: inventory holding cost per widget per month ($I=5$)  
$B$: back-order cost per widget per month ($B=10$)  
$P$: revenue per widget ($P=200$)  

**Decision Variables ($t \in T$):**  
$w_t$: workers in month $t$  
$h_t$: workers hired at the start of month $t$   
$l_t$: workers laid off at the start of month $t$  
$o_{t}$: overtime hours worked in month $t$  
$p_t$: widgets produced in month $t$  
$i_t$: inventory level at the end of month $t$  
$b_t$: back-orders for month $t$  

**Model Formulation:**

Objective function:
$$\min \sum_{t \in T} S w_t  + O o_t + H h_t + L l_t + C p_t + I i_t + B b_t$$

Subject to (Constraints):

Non-negativity constraint:
$$w_t, h_t, l_t, o_t, p_t, i_t, b_t \ge 0,~~ \forall t \in T$$

Worker constraint:
$$w_t = w_{(t-1)} + h_t - l_t,~~ \forall t \in T$$

Overtime hours constraint:
$$o_t \leq M w_t,~~ \forall t \in T$$

Production constraint:
$$3 p_t \leq R w_t + o_t,~~ \forall t \in T$$

Demand constraint:
$$p_t + i_{(t-1)} + b_t - i_t = D_t + b_{(t-1)},~~ \forall t \in T$$

Final back-orders constraint:
$$b_6 = 0$$

This *Final back-orders* constraint is needed, else, we can postpone satisfying demand until the end of the planning horizon, which could lead to strange answers (e.g., lay-off all workers and back-order everything).



In [23]:
# Problem data
import pulp

# Parameters
T = range(1, 7)  # Months 1 to 6
w_0 = 30         # Initial workers
b_0 = 0          # Initial back-orders
i_0 = 200        # Initial inventory
D_t = {1: 1200, 2: 1500, 3: 1800, 4: 1900, 5: 2100, 6: 2200}      # Dictionary of demand for each month t
S = 160 * 20     # Regular monthly pay rate per worker
O = 30           # Overtime pay rate per hour
H = 1000         # Cost of hiring a new worker
L = 2000         # Cost of laying off a worker
R = 160          # Regular time hours per worker per month
M = 20           # Max overtime hours per worker per month
C = 10           # Raw material cost per widget
I = 5            # Inventory holding cost per widget per month
B = 10           # Back-order cost per widget per month
P = 200          # Revenue per widget

In [24]:
# Create a LP Minimization problem
problem = pulp.LpProblem("WidgetProductionProblem", pulp.LpMinimize)

# Decision Variables
w = pulp.LpVariable.dicts("Workers", T, lowBound=0, cat='Integer')
h = pulp.LpVariable.dicts("Hired", T, lowBound=0, cat='Continuous')
l = pulp.LpVariable.dicts("LaidOff", T, lowBound=0, cat='Continuous')
o = pulp.LpVariable.dicts("OvertimeHours", T, lowBound=0, cat='Continuous')
p = pulp.LpVariable.dicts("Produced", T, lowBound=0, cat='Continuous')
i = pulp.LpVariable.dicts("Inventory", T, lowBound=0, cat='Continuous')
b = pulp.LpVariable.dicts("BackOrders", T, lowBound=0, cat='Continuous')

# Objective Function
problem += pulp.lpSum([S * w[t] + O * o[t] + H * h[t] + L * l[t] + C * p[t] + I * i[t] + B * b[t] for t in T])

# Constraints
# Worker constraint
problem += w[1] == w_0 + h[1] - l[1]
for t in T[1:]:
    problem += w[t] == w[t - 1] + h[t] - l[t]

# Overtime hours constraint
for t in T:
    problem += o[t] <= M * w[t]

# Production constraint
for t in T:
    problem += 3 * p[t] <= R * w[t] + o[t]

# Demand constraint
problem += p[1] + i_0 + b[1] - i[1] == D_t[1] + b_0
for t in T[1:]:
    problem += p[t] + i[t - 1] + b[t] - i[t] == D_t[t] + b[t - 1]

# Final back-orders constraint
problem += b[6] == 0

# Solve the problem
problem.solve()

# Output results
if problem.status == pulp.LpStatusOptimal:
    print("Solution Status: Optimal")
    print(f"Total Cost: ${pulp.value(problem.objective):.2f}")
    for t in T:
        print(f"Month {t}:")
        print(f" Workers: {pulp.value(w[t])}")
        print(f" Hired: {pulp.value(h[t])}")
        print(f" Laid off: {pulp.value(l[t])}")
        print(f" Overtime hours: {pulp.value(o[t])}")
        print(f" Widgets produced: {pulp.value(p[t])}")
        print(f" Inventory: {pulp.value(i[t])}")
        print(f" Back-orders: {pulp.value(b[t])}")
else:
    print("Problem not solved to optimality.")

Solution Status: Optimal
Total Cost: $754000.00
Month 1:
 Workers: 30.0
 Hired: 0.0
 Laid off: 0.0
 Overtime hours: 0.0
 Widgets produced: 1593.3333
 Inventory: 593.33333
 Back-orders: 0.0
Month 2:
 Workers: 30.0
 Hired: 0.0
 Laid off: 0.0
 Overtime hours: 0.0
 Widgets produced: 1600.0
 Inventory: 693.33333
 Back-orders: 0.0
Month 3:
 Workers: 30.0
 Hired: 0.0
 Laid off: 0.0
 Overtime hours: 0.0
 Widgets produced: 1600.0
 Inventory: 493.33333
 Back-orders: 0.0
Month 4:
 Workers: 35.0
 Hired: 5.0
 Laid off: 0.0
 Overtime hours: 0.0
 Widgets produced: 1866.6667
 Inventory: 460.0
 Back-orders: 0.0
Month 5:
 Workers: 36.0
 Hired: 1.0
 Laid off: 0.0
 Overtime hours: 0.0
 Widgets produced: 1920.0
 Inventory: 280.0
 Back-orders: 0.0
Month 6:
 Workers: 36.0
 Hired: 0.0
 Laid off: 0.0
 Overtime hours: 0.0
 Widgets produced: 1920.0
 Inventory: 0.0
 Back-orders: 0.0


In [25]:
# using ChatGTP 3.5 -

# Create LP problem
prob = LpProblem("Aggregate_Production_Plan", LpMinimize)

# Define decision variables
w = LpVariable.dicts("Workers", months, lowBound=0, cat='Integer')
h = LpVariable.dicts("Hired", months, lowBound=0, cat='Continuous')
l = LpVariable.dicts("Laid_off", months, lowBound=0, cat='Continuous')
o = LpVariable.dicts("Overtime", months, lowBound=0, cat='Continuous')
p = LpVariable.dicts("Production", months, lowBound=0, cat='Continuous')
i = LpVariable.dicts("Inventory", months, lowBound=0, cat='Continuous')
b = LpVariable.dicts("Backorder", months, lowBound=0, cat='Continuous')

# Define objective function
prob += lpSum([S * w[t] + O * o[t] + H * h[t] + L * l[t] + C * p[t] + I * i[t] + B * b[t] for t in months])

# Define constraints
for t in months:
    if t == 1:
        prob += w[t] == w_0 + h[t] - l[t]
    else:
        prob += w[t] == w[t - 1] + h[t] - l[t]
    prob += o[t] <= M * w[t]
    prob += 3 * p[t] <= R * w[t] + o[t]
    if t == 1:
        prob += p[t] + i_0 + b[t] - i[t] == demand[t]
    else:
        prob += p[t] + i[t - 1] + b[t] - i[t] == demand[t] + b[t - 1]
        

# Final back-orders constraint
prob += b[6] == 0

import textwrap
#print(textwrap.fill(str(prob), width=80))


# Solve the problem
prob.solve()

# Print the results
print("Status:", LpStatus[prob.status])
print("Total Cost:", value(prob.objective))
for t in months:
    print(f"Month {t}:")
    print(f"Workers: {int(w[t].varValue)}")
    print(f"Hired: {int(h[t].varValue)}")
    print(f"Laid Off: {int(l[t].varValue)}")
    print(f"Overtime: {int(o[t].varValue)}")
    print(f"Production: {int(p[t].varValue)}")
    print(f"Inventory: {int(i[t].varValue)}")
    print(f"Backorder: {int(b[t].varValue)}")
    print()



Status: Optimal
Total Cost: 753999.9999500001
Month 1:
Workers: 30
Hired: 0
Laid Off: 0
Overtime: 0
Production: 1593
Inventory: 593
Backorder: 0

Month 2:
Workers: 30
Hired: 0
Laid Off: 0
Overtime: 0
Production: 1600
Inventory: 693
Backorder: 0

Month 3:
Workers: 30
Hired: 0
Laid Off: 0
Overtime: 0
Production: 1600
Inventory: 493
Backorder: 0

Month 4:
Workers: 35
Hired: 5
Laid Off: 0
Overtime: 0
Production: 1866
Inventory: 460
Backorder: 0

Month 5:
Workers: 36
Hired: 1
Laid Off: 0
Overtime: 0
Production: 1920
Inventory: 280
Backorder: 0

Month 6:
Workers: 36
Hired: 0
Laid Off: 0
Overtime: 0
Production: 1920
Inventory: 0
Backorder: 0

